# `anato-mesh` in Jupyter Notebook 

`anato_mesh.py` contains the main functions for calculating the partition-level curvatures using python to reproduce the algorithm originally published by K. Khabaz here: https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1011815

In [2]:
from anato_mesh import *
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

In [3]:
#- Pocivavsek lab parent directory
parent_path = 'Z:\\aorta\\thoracic'
#- Cohort groups filter
group_str = ['AV', 'DJ', 'KK', 'KY', 'KT', 'KP', 'KR', 'SA']
#- Mesh formats filter
file_str = ['M5','M10']

| `quantities` | `point_removal` |
| --- | --- |
| 'Gaussian' | 'curvature' | 
| 'Casorati' | 'thoracic' | 
| 'IntMean' |
| 'Total' |
| 'ShapeIndex' |
| 'Willmore' |

You must specify at least one variable in `quantitites` and must also define the `point_removal` method. When in doubt, use the `curvature` method because it is applicable to all geometries. 

In [4]:
quantities = ['Casorati','Total']
point_removal = 'curvature'

The number of surface partitions for each surface is determined by the following equation. `m_set` provides an easily adjustable parameter for further partition scaling and allows you to calculate more than one scaling at once. The optimized value for the thoracic aorta TEVAR dataset is `m=1`.

$$partitions = m \times \left(\frac{SA}{R^2}\right)$$

In [5]:
m_set = [0.5, 1, 5]

In [6]:
results = GetAortaMeshResults(parent_path, group_str, file_str, point_removal, quantities, m_set)

Organizing paths and file names:
Adding AV1_1_M10.parquet
Adding AV1_1_M5.parquet
Adding AV1_5_M10.parquet
Adding AV1_5_M5.parquet
Adding AV12_2_M10.parquet
Adding AV12_2_M5.parquet
Adding AV12_7_M10.parquet
Adding AV12_7_M5.parquet
Adding AV13_1_M10.parquet
Adding AV13_1_M5.parquet
Adding AV13_4_M10.parquet
Adding AV13_4_M5.parquet
Adding AV16_1_M10.parquet
Adding AV16_1_M5.parquet
Adding AV16_5_M10.parquet
Adding AV16_5_M5.parquet
Adding AV16_6_M10.parquet
Adding AV16_6_M5.parquet
Adding AV18_1_M10.parquet
Adding AV18_1_M5.parquet
Adding AV18_6_M10.parquet
Adding AV18_6_M5.parquet
Adding AV19_1_M10.parquet
Adding AV19_1_M5.parquet
Adding AV19_3_M10.parquet
Adding AV19_3_M5.parquet
Adding AV2_1_M10.parquet
Adding AV2_1_M5.parquet
Adding AV2_3_M10.parquet
Adding AV2_3_M5.parquet
Adding AV20_1_M10.parquet
Adding AV20_1_M5.parquet
Adding AV20_3_M10.parquet
Adding AV20_3_M5.parquet
Adding AV20_9_M10.parquet
Adding AV20_9_M5.parquet
Adding AV21_1_M10.parquet
Adding AV21_1_M5.parquet
Adding

Adding DJ21_2_M10.parquet
Adding DJ21_2_M5.parquet
Adding DJ21_3_M10.parquet
Adding DJ21_3_M5.parquet
Adding DJ22_1_M10.parquet
Adding DJ22_1_M5.parquet
Adding DJ22_10_M10.parquet
Adding DJ22_10_M5.parquet
Adding DJ22_11_M10.parquet
Adding DJ22_11_M5.parquet
Adding DJ22_13_M10.parquet
Adding DJ22_13_M5.parquet
Adding DJ22_2_M10.parquet
Adding DJ22_2_M5.parquet
Adding DJ22_3_M10.parquet
Adding DJ22_3_M5.parquet
Adding DJ22_4_M10.parquet
Adding DJ22_4_M5.parquet
Adding DJ22_5_M10.parquet
Adding DJ22_5_M5.parquet
Adding DJ22_6_M10.parquet
Adding DJ22_6_M5.parquet
Adding DJ22_8_M10.parquet
Adding DJ22_8_M5.parquet
Adding DJ22_9_M10.parquet
Adding DJ22_9_M5.parquet
Adding DJ23_10_M10.parquet
Adding DJ23_10_M5.parquet
Adding DJ23_12_M10.parquet
Adding DJ23_12_M5.parquet
Adding DJ23_2_M10.parquet
Adding DJ23_2_M5.parquet
Adding DJ23_3_M10.parquet
Adding DJ23_3_M5.parquet
Adding DJ23_4_M10.parquet
Adding DJ23_4_M5.parquet
Adding DJ23_5_M10.parquet
Adding DJ23_5_M5.parquet
Adding DJ23_6_M10.par

Adding KK13_5_M10.parquet
Adding KK13_5_M5.parquet
Adding KK14_1_M10.parquet
Adding KK14_1_M5.parquet
Adding KK14_2_M10.parquet
Adding KK14_2_M5.parquet
Adding KK14_3_M10.parquet
Adding KK14_3_M5.parquet
Adding KK14_4_M10.parquet
Adding KK14_4_M5.parquet
Adding KK14_5_M10.parquet
Adding KK14_5_M5.parquet
Adding KK14_6_M10.parquet
Adding KK14_6_M5.parquet
Adding KK14_7_M10.parquet
Adding KK14_7_M5.parquet
Adding KK14_8_M10.parquet
Adding KK14_8_M5.parquet
Adding KK15_1_M10.parquet
Adding KK15_1_M5.parquet
Adding KK15_2_M10.parquet
Adding KK15_2_M5.parquet
Adding KK15_3_M10.parquet
Adding KK15_3_M5.parquet
Adding KK16_1_M10.parquet
Adding KK16_1_M5.parquet
Adding KK16_2_M10.parquet
Adding KK16_2_M5.parquet
Adding KK16_3_M10.parquet
Adding KK16_3_M5.parquet
Adding KK16_4_M10.parquet
Adding KK16_4_M5.parquet
Adding KK16_5_M10.parquet
Adding KK16_5_M5.parquet
Adding KK16_6_M10.parquet
Adding KK16_6_M5.parquet
Adding KK16_8_M10.parquet
Adding KK16_8_M5.parquet
Adding KK16_9_M10.parquet
Addin

Adding KY112_1_M10.parquet
Adding KY112_1_M5.parquet
Adding KY112_2_M10.parquet
Adding KY112_2_M5.parquet
Adding KY113_1_M10.parquet
Adding KY113_1_M5.parquet
Adding KY114_1_M10.parquet
Adding KY114_1_M5.parquet
Adding KY115_1_M10.parquet
Adding KY115_1_M5.parquet
Adding KY116_1_M10.parquet
Adding KY116_1_M5.parquet
Adding KY116_2_M10.parquet
Adding KY116_2_M5.parquet
Adding KY117_1_M10.parquet
Adding KY117_1_M5.parquet
Adding KY118_1_M10.parquet
Adding KY118_1_M5.parquet
Adding KY118_2_M10.parquet
Adding KY118_2_M5.parquet
Adding KY119_1_M10.parquet
Adding KY119_1_M5.parquet
Adding KY12_1_M10.parquet
Adding KY12_1_M5.parquet
Adding KY12_2_M10.parquet
Adding KY12_2_M5.parquet
Adding KY120_1_M10.parquet
Adding KY120_1_M5.parquet
Adding KY120_2_M10.parquet
Adding KY120_2_M5.parquet
Adding KY121_1_M10.parquet
Adding KY121_1_M5.parquet
Adding KY122_1_M10.parquet
Adding KY122_1_M5.parquet
Adding KY122_2_M10.parquet
Adding KY122_2_M5.parquet
Adding KY123_1_M10.parquet
Adding KY123_1_M5.parqu

Adding KY31_2_M10.parquet
Adding KY31_2_M5.parquet
Adding KY32_1_M10.parquet
Adding KY32_1_M5.parquet
Adding KY32_2_M10.parquet
Adding KY32_2_M5.parquet
Adding KY33_1_M10.parquet
Adding KY33_1_M5.parquet
Adding KY33_2_M10.parquet
Adding KY33_2_M5.parquet
Adding KY34_1_M10.parquet
Adding KY34_1_M5.parquet
Adding KY34_2_M10.parquet
Adding KY34_2_M5.parquet
Adding KY35_1_M10.parquet
Adding KY35_1_M5.parquet
Adding KY35_2_M10.parquet
Adding KY35_2_M5.parquet
Adding KY36_1_M10.parquet
Adding KY36_1_M5.parquet
Adding KY36_2_M10.parquet
Adding KY36_2_M5.parquet
Adding KY37_1_M10.parquet
Adding KY37_1_M5.parquet
Adding KY37_2_M10.parquet
Adding KY37_2_M5.parquet
Adding KY38_1_M10.parquet
Adding KY38_1_M5.parquet
Adding KY38_2_M10.parquet
Adding KY38_2_M5.parquet
Adding KY39_1_M10.parquet
Adding KY39_1_M5.parquet
Adding KY39_2_M10.parquet
Adding KY39_2_M5.parquet
Adding KY4_1_M10.parquet
Adding KY4_1_M5.parquet
Adding KY4_2_M10.parquet
Adding KY4_2_M5.parquet
Adding KY40_1_M10.parquet
Adding KY

...

K-Means done.
Quantities calculated.
KR60_9_M10.parquet with 625 patches using m=5 took: 00:00:00
The curvature point removal method was employed.
K-Means done.
Quantities calculated.
KR60_9_M5.parquet with 625 patches using m=5 took: 00:00:00
The curvature point removal method was employed.
K-Means done.
Quantities calculated.
KR61_1_M10.parquet with 780 patches using m=5 took: 00:00:00
The curvature point removal method was employed.
K-Means done.
Quantities calculated.
KR61_1_M5.parquet with 780 patches using m=5 took: 00:00:00
The curvature point removal method was employed.
K-Means done.
Quantities calculated.
KR61_2_M10.parquet with 610 patches using m=5 took: 00:00:00
The curvature point removal method was employed.
K-Means done.
Quantities calculated.
KR61_2_M5.parquet with 610 patches using m=5 took: 00:00:00
The curvature point removal method was employed.
K-Means done.
Quantities calculated.
KR62_1_M10.parquet with 685 patches using m=5 took: 00:00:00
The curvature point rem

K-Means done.
Quantities calculated.
KR68_2_M10.parquet with 650 patches using m=5 took: 00:00:00
The curvature point removal method was employed.
K-Means done.
Quantities calculated.
KR68_2_M5.parquet with 650 patches using m=5 took: 00:00:00
The curvature point removal method was employed.
K-Means done.
Quantities calculated.
KR68_3_M10.parquet with 605 patches using m=5 took: 00:00:00
The curvature point removal method was employed.
K-Means done.
Quantities calculated.
KR68_3_M5.parquet with 610 patches using m=5 took: 00:00:00
The curvature point removal method was employed.
K-Means done.
Quantities calculated.
KR68_4_M10.parquet with 600 patches using m=5 took: 00:00:00
The curvature point removal method was employed.
K-Means done.
Quantities calculated.
KR68_4_M5.parquet with 600 patches using m=5 took: 00:00:00
The curvature point removal method was employed.
K-Means done.
Quantities calculated.
KR69_1_M10.parquet with 610 patches using m=5 took: 00:00:00
The curvature point rem

K-Means done.
Quantities calculated.
KR74_4_M10.parquet with 390 patches using m=5 took: 00:00:00
The curvature point removal method was employed.
K-Means done.
Quantities calculated.
KR74_4_M5.parquet with 390 patches using m=5 took: 00:00:00
The curvature point removal method was employed.
K-Means done.
Quantities calculated.
KR75_1_M10.parquet with 700 patches using m=5 took: 00:00:00
The curvature point removal method was employed.
K-Means done.
Quantities calculated.
KR75_1_M5.parquet with 700 patches using m=5 took: 00:00:00
The curvature point removal method was employed.
K-Means done.
Quantities calculated.
KR76_1_M10.parquet with 570 patches using m=5 took: 00:00:00
The curvature point removal method was employed.
K-Means done.
Quantities calculated.
KR76_1_M5.parquet with 560 patches using m=5 took: 00:00:00
The curvature point removal method was employed.
K-Means done.
Quantities calculated.
KR76_2_M10.parquet with 555 patches using m=5 took: 00:00:00
The curvature point rem

K-Means done.
Quantities calculated.
SA18_1_M10.parquet with 865 patches using m=5 took: 00:00:00
The curvature point removal method was employed.
K-Means done.
Quantities calculated.
SA18_1_M5.parquet with 870 patches using m=5 took: 00:00:01
The curvature point removal method was employed.
K-Means done.
Quantities calculated.
SA19_1_M10.parquet with 795 patches using m=5 took: 00:00:00
The curvature point removal method was employed.
K-Means done.
Quantities calculated.
SA19_1_M5.parquet with 800 patches using m=5 took: 00:00:00
The curvature point removal method was employed.
K-Means done.
Quantities calculated.
SA2_1_M10.parquet with 852 patches using m=5 took: 00:00:00
The curvature point removal method was employed.
K-Means done.
Quantities calculated.
SA2_1_M5.parquet with 985 patches using m=5 took: 00:00:00
The curvature point removal method was employed.
K-Means done.
Quantities calculated.
SA20_1_M10.parquet with 1370 patches using m=5 took: 00:00:01
The curvature point remo

K-Means done.
Quantities calculated.
SA43_1_M10.parquet with 890 patches using m=5 took: 00:00:00
The curvature point removal method was employed.
K-Means done.
Quantities calculated.
SA43_1_M5.parquet with 875 patches using m=5 took: 00:00:00
The curvature point removal method was employed.
K-Means done.
Quantities calculated.
SA44_1_M10.parquet with 735 patches using m=5 took: 00:00:00
The curvature point removal method was employed.
K-Means done.
Quantities calculated.
SA44_1_M5.parquet with 745 patches using m=5 took: 00:00:00
The curvature point removal method was employed.
K-Means done.
Quantities calculated.
SA45_1_M10.parquet with 850 patches using m=5 took: 00:00:00
The curvature point removal method was employed.
K-Means done.
Quantities calculated.
SA45_1_M5.parquet with 860 patches using m=5 took: 00:00:00
The curvature point removal method was employed.
K-Means done.
Quantities calculated.
SA46_1_M10.parquet with 875 patches using m=5 took: 00:00:00
The curvature point rem

In [8]:
results

,Scan_Name,AvElemPatch,AvElemArea,AvPatchArea,Num_Patches,Casorati_Mean,Casorati_Var,Casorati_Fluct,Casorati_Sum,Total_Mean,...,Total_Fluct,Total_Sum,Mesh_Area,Mesh_Volume,Mean_Edge_Length,Mean_Face_Angle,Mean_Vertex_Angle,Euler_Number,Moment_Inertia_Mag,Partition_Prefactor
0,AV1_1_M10,67.422535,4.337277,291.655463,71,0.061050,0.000060,0.000061,4.334544,0.059844,...,0.140448,4.248916,21502.531307,122730.108572,3.231947,0.126455,1.047198,2.0,6.458261e+08,0.5
1,AV1_1_M5,137.028571,2.176212,298.012881,70,0.061054,0.000058,0.000057,4.273810,0.043902,...,0.141907,3.073117,21552.807962,123417.450376,2.289632,0.089896,1.047198,2.0,6.502650e+08,0.5
2,AV1_5_M10,88.902778,4.352549,386.482697,72,0.052884,0.000062,0.000061,3.807624,0.070562,...,0.117449,5.080435,28709.137785,192971.779847,3.238793,0.109016,1.047198,2.0,1.425914e+09,0.5
3,AV1_5_M5,178.027778,2.187350,389.000337,72,0.053396,0.000058,0.000057,3.844527,0.059599,...,0.136877,4.291120,28752.040053,193787.061639,2.297816,0.077141,1.047198,2.0,1.433845e+09,0.5
4,AV12_2_M10,68.012048,4.338235,294.374724,83,0.058389,0.000046,0.000044,4.846300,0.050588,...,0.100118,4.198794,25497.889166,153054.649201,3.234964,0.118936,1.047198,2.0,1.186899e+09,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8665,SA7_1_M5,9.499454,2.181228,20.598164,915,0.067414,0.000229,0.000215,61.684267,0.002535,...,0.001305,2.319465,19685.066854,103840.238928,2.288267,0.098111,1.047198,2.0,5.117702e+08,5
8666,SA8_1_M10,6.984722,4.367467,30.291321,720,0.057514,0.000076,0.000076,41.410220,0.002268,...,0.001636,1.633016,23136.152742,138335.376649,3.237787,0.121083,1.047198,2.0,8.056780e+08,5
8667,SA8_1_M5,14.115278,2.178061,30.605452,720,0.057192,0.000087,0.000094,41.178153,0.000757,...,0.001526,0.544814,23180.471656,139055.021092,2.288252,0.084103,1.047198,2.0,8.109090e+08,5
8668,SA9_1_M10,4.007609,4.366657,17.326829,920,0.071983,0.000216,0.000239,66.224733,0.003232,...,0.001214,2.973124,17408.228617,79265.511477,3.234060,0.156214,1.047198,4.0,3.988945e+08,5


In [9]:
import plotly.express as px

In [10]:
results_norm = GroupsNormalize(SplitUpScanName(results), 'Casorati_Mean', 'Total_Fluct', 'KY')

In [11]:
#- Meta data integration 
directory = 'Z:\\aorta\\meta_data'
file_name = 'MasterMetaData.xlsx'
#- Cohort meta data group names
cohort_list = ['UC_NORMAL', 'UC_PEDS', 'UC_TEVAR', 'ENDOSPAN', 'MEDTRONIC', 'GORE_801', 'GORE_802', 'GORE_803']
#- Categorical labels 
cat_columns = ['Label', 'Outcome']

In [12]:
results_meta = MergeMetaData(directory, file_name, cohort_list, results_norm, cat_columns)

In [22]:
df_preop = results_meta[results_meta['Preop']==1]

In [27]:
fig = px.scatter(results_meta, x='Mean_Radius', y='Total_Fluct_Norm', color='Label', hover_data=['Scan_Name'])
fig.update_layout(yaxis_range=[0,7])
fig.show()

In [15]:
SaveToXLSX('Z:\\aorta\\projects', 'all_scale_sampled.xlsx', results_meta)